In [2]:
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

pd.options.display.max_colwidth = 100

In [3]:
# substitua o caminho do arquivo pelo caminho do seu arquivo .csv nos seus documentos
caminho_arquivo = 'C:/Users/JOSE/2022-10-03.csv'

# use a função read_csv para ler o arquivo e armazená-lo em um DataFrame
df = pd.read_csv(caminho_arquivo)

# imprima o DataFrame para verificar se o arquivo foi lido corretamente
print(df)

C:\Users\JOSE\AppData\Local\Temp\ipykernel_9972\3871414642.py:5: DtypeWarning:

Columns (26,100,101) have mixed types. Specify dtype option on import or set low_memory=False.



        id_cotacao                           grupo_calculo   preco_id  \
0         13296967  0013296967-1664831874-5127bd118ec1b3f9  327535619   
1         13296967  0013296967-1664831874-5127bd118ec1b3f9  327535618   
2         13296967  0013296967-1664831874-5127bd118ec1b3f9  327535752   
3         13296967  0013296967-1664831874-5127bd118ec1b3f9  327535661   
4         13296967  0013296967-1664831874-5127bd118ec1b3f9  327535721   
...            ...                                     ...        ...   
249774    18732356  0018732356-1664852243-3e8f69f43bbb364b  327556565   
249775    18732356  0018732356-1664852243-3e8f69f43bbb364b  327556567   
249776    18732356  0018732356-1664852243-3e8f69f43bbb364b  327556566   
249777    18732356  0018732356-1664852243-3e8f69f43bbb364b  327556569   
249778    18732356  0018732356-1664852243-3e8f69f43bbb364b  327556560   

        cobertura_id  usuario_id  corretora_id  cliente_id  ranking  \
0           16830520       14688          7799     1

In [4]:
df.head(3)

,id_cotacao,grupo_calculo,preco_id,cobertura_id,usuario_id,corretora_id,cliente_id,ranking,REFERENCIA,tempo,...,danos_morais_premio,isencao_franquia,franquia_retorno,franquia_valor,comissao,comissao_retorno,msg,msg_ajuste,Unnamed: 100,Unnamed: 101
0,13296967,0013296967-1664831874-5127bd118ec1b3f9,327535619,16830520,14688,7799,1392675,1,61,3,...,NaN,False,reduzida50,0,15,15,<b>Cobertura:</b> Exclusiva para Indenização Integral por Roubo/Furto<br><b>Exclusões:</b> Não h...,Fipe ajustado para 95% conforme validação da CIA<br><b>Assistência 24Hrs/Reboque</b> foi ajustad...,NaN,NaN
1,13296967,0013296967-1664831874-5127bd118ec1b3f9,327535618,16830520,14688,7799,1392675,2,61,3,...,NaN,False,reduzida50,0,15,15,<b>Cobertura:</b> Exclusiva para Indenização Integral por Roubo/Furto<br><b>Exclusões:</b> Não h...,Fipe ajustado para 95% conforme validação da CIA<br><b>Assistência 24Hrs/Reboque</b> foi ajustad...,NaN,NaN
2,13296967,0013296967-1664831874-5127bd118ec1b3f9,327535752,16830520,14688,7799,1392675,3,27,27,...,"16,53",False,reduzida50,"5584,59",15,15,As coberturas <b>Carta Verde</b> e <b>Extensão de Perímetro</b> foram contratadas por padrão par...,"<b>Atenção!</b> A vigência do seguro informada é inválida, ajustada para 03/10/2022 à 03/10/2023...",NaN,NaN


In [5]:
# Obtém a quantidade total de linhas no DataFrame
total_linhas = df.shape[0]

# Obtém a quantidade de valores distintos em cada coluna
quantidade_distintos = df.nunique()

print(f'Quantidade total de linhas: {total_linhas}')
print('Quantidade de valores distintos por coluna:')
print(quantidade_distintos)

Quantidade total de linhas: 249779
Quantidade de valores distintos por coluna:
id_cotacao           14029
grupo_calculo        21301
preco_id            249779
cobertura_id         14919
usuario_id            3204
                     ...  
comissao_retorno        32
msg                   1746
msg_ajuste            3736
Unnamed: 100             5
Unnamed: 101             4
Length: 102, dtype: int64


In [6]:
df.columns

Index(['id_cotacao', 'grupo_calculo', 'preco_id', 'cobertura_id', 'usuario_id',
       'corretora_id', 'cliente_id', 'ranking', 'REFERENCIA', 'tempo',
       ...
       'danos_morais_premio', 'isencao_franquia', 'franquia_retorno',
       'franquia_valor', 'comissao', 'comissao_retorno', 'msg', 'msg_ajuste',
       'Unnamed: 100', 'Unnamed: 101'],
      dtype='object', length=102)

In [7]:
#Aqui descobrimos que nossa coluna "preco_id" é a chave única
#Agora vou começar a simplificar as colunas, juntar assistências, coberturas, etc etc
#Vou criar um novo DF com o básico e vou enriquecendo conforme for encontrando as colunas necessárias

df_filtered = df.filter(items = ['id_cotacao',
                                 'preco_id',
                                 'REFERENCIA',
                                 'tiposeguro',
                                 'seguradora_renovacao',
                                 'renovacao_propria',
                                 'inicio_vigencia',
                                 'termino_vigencia', #criar coluna com vigência
                                 'tipo_cobertura_retorno',
                                 'franquia_retorno',
                                 'coberturas_vidros_retorno',  #criar coluna sim/não
                                 'coberturas_17a25anos',
                                 'cobertura_para_outro_condutor',
                                 'coberturas_assistencia',
                                 'carro_reserva_retorno', #criar sim/não
                                 'perfil_carro_reserva',
                                 'assistencia_24h_retorno', #criar sim/não com "Nâo Contratado"
                                 'dias_carro_reserva',
                                 'segurado_sexo',
                                 'segurado_data_nascimento', #criar coluna idade
                                 'codigo_fipe',
                                 'fabricante',
                                 'zero_km',
                                 'percentual_fipe_retorno',
                                 'marcas_tipo',
                                 'tipo_veiculo',
                                 'tipo_combustivel', #criar coluna kit_gas
                                 'veiculo_blindado',
                                 'veiculo_retorno',
                                 'bonus_atual', #criar categoria 1a3, 4a7, 8a10
                                 'cep_circula', #criar coluna regiao =SE(NÚM.CARACT([@[cep_circula]])=8;ESQUERDA([@[cep_circula]];2);ESQUERDA([@[cep_circula]];1))
                                 'utilizacao_veiculo',
                                 'premio_bruto',
                                 'premio_liquido',
                                 'comissao_retorno'])


df_filtered.head(3)

,id_cotacao,preco_id,REFERENCIA,tiposeguro,seguradora_renovacao,renovacao_propria,inicio_vigencia,termino_vigencia,tipo_cobertura_retorno,franquia_retorno,...,tipo_veiculo,tipo_combustivel,veiculo_blindado,veiculo_retorno,bonus_atual,cep_circula,utilizacao_veiculo,premio_bruto,premio_liquido,comissao_retorno
0,13296967,327535619,61,Renovacao,allianz,Não,26/01/2021,26/01/2022,Roubo e Furto,reduzida50,...,particular,FLEX,Nao,Corolla Altis Prem. 1.8 Aut. (Hibrido),5,38412252,pessoal,"1823,99","1698,63",15
1,13296967,327535618,61,Renovacao,allianz,Não,26/01/2021,26/01/2022,Compreensiva,reduzida50,...,particular,FLEX,Nao,Corolla Altis Prem. 1.8 Aut. (Hibrido),5,38412252,pessoal,"3068,24","2857,37",15
2,13296967,327535752,27,Renovacao,allianz,Não,26/01/2021,26/01/2022,"Colisao/Incendio, Roubo e Furto",reduzida50,...,particular,FLEX,Nao,Toyota - Corolla Altis Prem. 1.8 Aut. (Híbrido),5,38412252,pessoal,"4228,65","3938,02",15


In [9]:
#Agora vou criar as colunas que marquei na tabela à cima
#Preservando o df filtrado
df_improved = df_filtered

#criar coluna com vigência
df_improved['inicio_vigencia'] = pd.to_datetime(df_improved['inicio_vigencia'], errors = 'coerce', infer_datetime_format=True)
df_improved['termino_vigencia'] = pd.to_datetime(df_improved['termino_vigencia'], errors = 'coerce', infer_datetime_format=True)
df_improved = df_improved.assign(date_diff=(df_improved['termino_vigencia'] - df_improved['inicio_vigencia']).dt.days)

In [12]:
#criar coluna com sim/não para Vidros e Carro Reserva
def verifica_contratacao(valor):
    if valor == 'Não contratado' or 'NÃ£o possui Cobertura' or 'nÃ£o Contratada' or 'NÃ£o Contempla' or 'NÃ£o contemplado' or 'NÃ£o Contratado' or 'Sem carro reserva	':
        return 'Não'
    elif isinstance(valor, str) and 'não' in valor.lower(): 
        return 'Não'
    elif isinstance(valor, str) and 'sem' in valor.lower(): 
        return 'Não'
    elif pd.isna(valor):
        return 'Não'
    else:
        return 'Sim'


# Aplica a função na coluna e cria uma nova coluna nova coluna com o resultado
df_improved['vidros_sim'] = df_improved['coberturas_vidros_retorno'].apply(verifica_contratacao)
df_improved['carro_reserva_sim'] = df_improved['carro_reserva_retorno'].apply(verifica_contratacao)
df_improved['assistencia_24h_sim'] = df_improved['assistencia_24h_retorno'].apply(verifica_contratacao)

#se as qualquer assistência for sim, então marca "Sim", se as 3 forem não, então marca "Não"
def validar_assistencias(row):
    if 'Sim' in row.values:  # Se 'Sim' estiver presente em qualquer coluna
        return 'Sim'
    else:
        return 'Não'

# Aplicar a função em cada linha do DataFrame
df_improved['assistencias_total'] = df_improved[['vidros_sim', 'carro_reserva_sim', 'assistencia_24h_sim']].apply(validar_assistencias, axis=1)

# validando
df_improved[['coberturas_vidros_retorno', 'vidros_sim', 'carro_reserva_retorno', 'carro_reserva_sim', 'assistencia_24h_retorno', 'assistencia_24h_sim', 'assistencias_total']].head(10)

,coberturas_vidros_retorno,vidros_sim,carro_reserva_retorno,carro_reserva_sim,assistencia_24h_retorno,assistencia_24h_sim,assistencias_total
0,Não contratado,Não,Não contratado,Não,"200KM, Taxi, Chaveiro, Troca de Pneus",Não,Não
1,Não contratado,Não,Não contratado,Não,"200KM, Taxi, Chaveiro, Troca de Pneus",Não,Não
2,Vidros - Cobertura Completa,Não,NaN,Não,Guincho 400 km,Não,Não
3,"HDI Auto Vidros, Faróis, Lanternas, Retrovisores e Auxiliar",Não,Sem carro reserva,Não,Guincho 200 KM,Não,Não
4,Completo Rede Referenciada,Não,Carro Reserva não coletado,Não,Completo 500km,Não,Não
5,Vidro Protegido Plus (024),Não,Não desejo contratar,Não,Assist Dia/Noite 200Km (043),Não,Não
6,Vidro Protegido (025),Não,Auto Reserva 07 Dias (060),Não,Assist Dia/Noite 400Km (113),Não,Não
7,Vidros Completo - Rede Referenciada,Não,Não Contratado,Não,Assist. 24h Completo - 200km,Não,Não
8,"HDI Auto Vidros, Faróis, Lanternas, Retrovisores e Auxiliar",Não,Sem carro reserva,Não,Guincho 200 KM,Não,Não
9,Vidros Completa,Não,07 dias com ar condicionado,Não,Reboque Adicional de 200 Km,Não,Não


In [13]:
#preservando o df da celula anterior
df_improved2 = df_improved

#'segurado_data_nascimento', #criar coluna idade
# Converte a coluna de data_nascimento para o tipo datetime
df_improved2['segurado_data_nascimento'] = pd.to_datetime(df_improved2['segurado_data_nascimento'], infer_datetime_format=True)

# Calcula a idade com base na data atual
data_atual = datetime.now()
df_improved2['idade'] = (data_atual - df_improved2['segurado_data_nascimento']).astype('<m8[Y]')

#'idade', #criar faixa de edade
def faixa_idade_range(valor):
    if valor == 0:
        return '0'
    elif valor < 20:
        return '1a19'
    elif valor < 30:
        return '20a29'
    elif valor < 40:
        return '30a39'
    elif valor < 50:
        return '40a49'
    elif valor < 60:
        return '50a59'
    elif valor < 70:
        return '60a69'
    else:
        return '+70'
    
df_improved2['faixa_idade'] = df_improved2['idade'].apply(faixa_idade_range)

def clean_segurado_sexo(sexo):
    if sexo == 'female':
        return 'F'
    else:
        return sexo
    
df_improved2['segurado_sexo'] = df_improved2['segurado_sexo'].apply(clean_segurado_sexo)

#'tipo_combustivel', #criar coluna kit_gas
def verifica_gnv(valor):
    if isinstance(valor, str) and 'gnv' in valor.lower(): 
        return 'Sim'
    else:
        return 'Não'

# Aplica a função na coluna e cria uma nova coluna nova coluna com o resultado
df_improved2['kit_gas'] = df_improved2['tipo_combustivel'].apply(verifica_gnv)

#'bonus_atual', #criar categoria 1a3, 4a7, 8a10
def classe_bonus_range(valor):
    if valor == 0:
        return '0'
    elif valor < 4:
        return '1a3'
    elif valor < 8:
        return '4a7'
    else:
        return '8a10'
    
df_improved2['faixa_clase_bonus'] = df_improved2['bonus_atual'].apply(classe_bonus_range)
 
    
#'cep_circula', #criar coluna regiao =SE(NÚM.CARACT([@[cep_circula]])=8;ESQUERDA([@[cep_circula]];2);ESQUERDA([@[cep_circula]];1))
# Função para aplicar a lógica do CEP
def extrair_codigo_regiao(cep):
    cep_str = str(cep)  # Converter o valor para string
    if len(cep_str) == 8:
        return cep_str[:2]
    else:
        return cep_str[:1]

# Aplicar a função na coluna "cep_circula" e criar uma nova coluna "codigo_regiao"
df_improved2['codigo_regiao'] = df_improved2['cep_circula'].apply(extrair_codigo_regiao)

#validando
df_improved2[['tipo_combustivel','kit_gas','bonus_atual','faixa_clase_bonus', 'cep_circula', 'codigo_regiao', 'idade', 'faixa_idade']].head(3)

C:\Users\JOSE\AppData\Local\Temp\ipykernel_9972\211875375.py:6: UserWarning:

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.



,tipo_combustivel,kit_gas,bonus_atual,faixa_clase_bonus,cep_circula,codigo_regiao,idade,faixa_idade
0,FLEX,Não,5,4a7,38412252,38,39.0,30a39
1,FLEX,Não,5,4a7,38412252,38,39.0,30a39
2,FLEX,Não,5,4a7,38412252,38,39.0,30a39


In [14]:
df_improved2=df_improved
df_improved2['premio_bruto'] = df_improved['premio_bruto'].str.replace(',', '.').astype(float)
df_improved2.head(2)

,id_cotacao,preco_id,REFERENCIA,tiposeguro,seguradora_renovacao,renovacao_propria,inicio_vigencia,termino_vigencia,tipo_cobertura_retorno,franquia_retorno,...,date_diff,vidros_sim,carro_reserva_sim,assistencia_24h_sim,assistencias_total,idade,faixa_idade,kit_gas,faixa_clase_bonus,codigo_regiao
0,13296967,327535619,61,Renovacao,allianz,Não,2021-01-26,2022-01-26,Roubo e Furto,reduzida50,...,365.0,Não,Não,Não,Não,39.0,30a39,Não,4a7,38
1,13296967,327535618,61,Renovacao,allianz,Não,2021-01-26,2022-01-26,Compreensiva,reduzida50,...,365.0,Não,Não,Não,Não,39.0,30a39,Não,4a7,38


In [15]:
#criando premio ajustado
df_improved2['premio_ajustado'] = df_improved2['premio_bruto'] * (1 - (df_improved2['comissao_retorno'] / 100)) / (df_improved2['percentual_fipe_retorno'] / 100)

In [16]:
#agora vamos filtrar de novo o dataframe, pra deixar só as que queremos:
df_filtered2 = df_improved2.filter(items = ['id_cotacao',
                                 'preco_id',
                                 'REFERENCIA',
                                 'tiposeguro',
                                 'seguradora_renovacao',
                                 'renovacao_propria',
                                 #'inicio_vigencia',
                                 #'termino_vigencia',
                                 'date_diff', #criada coluna com vigência
                                 'tipo_cobertura_retorno',
                                 'franquia_retorno',
                                 'coberturas_17a25anos',
                                 'cobertura_para_outro_condutor',
                                 'coberturas_assistencia',
                                 'perfil_carro_reserva',
                                 'assistencias_km',
                                 #'carro_reserva_retorno',
                                 #'coberturas_vidros_retorno',
                                 #'assistencia_24h_retorno',
                                 'assistencias_total', #criada nova coluna para tem assistência ou não
                                 'dias_carro_reserva',
                                 'segurado_sexo',
                                 'fabricante',
                                 #'segurado_data_nascimento',
                                 'idade', #criada coluna idade
                                 'faixa_idade', #rango de edade         
                                 'codigo_fipe',
                                 'ano_fab',
                                 'ano_modelo',
                                 'zero_km',
                                 'percentual_fipe_retorno',
                                 'marcas_tipo',
                                 'tipo_veiculo',
                                 #'tipo_combustivel',
                                 'kit_gas', #criada coluna para quem tem kit_gas
                                 'veiculo_blindado',
                                 'veiculo_retorno',
                                 'bonus_atual',
                                 'faixa_clase_bonus', #criada categoria da classe de bonus, porem nao removi a coluna anterior
                                 #'cep_circula', 
                                 'codigo_regiao', #criada coluna com regiao do CEP
                                 'utilizacao_veiculo',
                                 'premio_bruto',
                                 'premio_liquido',
                                 'comissao_retorno',
                                 'premio_ajustado'])

df_filtered2.head(5)

,id_cotacao,preco_id,REFERENCIA,tiposeguro,seguradora_renovacao,renovacao_propria,date_diff,tipo_cobertura_retorno,franquia_retorno,coberturas_17a25anos,...,veiculo_blindado,veiculo_retorno,bonus_atual,faixa_clase_bonus,codigo_regiao,utilizacao_veiculo,premio_bruto,premio_liquido,comissao_retorno,premio_ajustado
0,13296967,327535619,61,Renovacao,allianz,Não,365.0,Roubo e Furto,reduzida50,Não,...,Nao,Corolla Altis Prem. 1.8 Aut. (Hibrido),5,4a7,38,pessoal,1823.99,"1698,63",15,1631.991053
1,13296967,327535618,61,Renovacao,allianz,Não,365.0,Compreensiva,reduzida50,Não,...,Nao,Corolla Altis Prem. 1.8 Aut. (Hibrido),5,4a7,38,pessoal,3068.24,"2857,37",15,2745.267368
2,13296967,327535752,27,Renovacao,allianz,Não,365.0,"Colisao/Incendio, Roubo e Furto",reduzida50,Não,...,Nao,Toyota - Corolla Altis Prem. 1.8 Aut. (Híbrido),5,4a7,38,pessoal,4228.65,"3938,02",15,3594.352500
3,13296967,327535661,18,Renovacao,allianz,Não,365.0,Compreensiva,reduzida50,Não,...,Nao,Corolla Altis Prem. Hybrid 1.8 Flex Aut,5,4a7,38,pessoal,6112.77,"5692,65",15,5195.854500
4,13296967,327535721,22,Renovacao,allianz,Não,365.0,Compreensiva,reduzida50,Não,...,Nao,NaN,5,4a7,38,pessoal,6522.99,"6074,68",15,5544.541500


In [17]:
df_filtered2.columns

Index(['id_cotacao', 'preco_id', 'REFERENCIA', 'tiposeguro',
       'seguradora_renovacao', 'renovacao_propria', 'date_diff',
       'tipo_cobertura_retorno', 'franquia_retorno', 'coberturas_17a25anos',
       'cobertura_para_outro_condutor', 'coberturas_assistencia',
       'perfil_carro_reserva', 'assistencias_total', 'dias_carro_reserva',
       'segurado_sexo', 'fabricante', 'idade', 'faixa_idade', 'codigo_fipe',
       'zero_km', 'percentual_fipe_retorno', 'marcas_tipo', 'tipo_veiculo',
       'kit_gas', 'veiculo_blindado', 'veiculo_retorno', 'bonus_atual',
       'faixa_clase_bonus', 'codigo_regiao', 'utilizacao_veiculo',
       'premio_bruto', 'premio_liquido', 'comissao_retorno',
       'premio_ajustado'],
      dtype='object')